In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999

In [2]:
def load_original_data():
    return pd.read_csv("data/Data_Marketing_Customer_Analysis_Round2.csv" )

In [ ]:
def clean_data(df):
    df.columns=[e.lower().replace(' ', '_') for e in df.columns]
    df=df.drop(columns=['unnamed:_0', 'vehicle_type',"customer","number_of_open_complaints"])
    df=df.dropna()
    df['effective_to_date']=pd.to_datetime(df['effective_to_date'], errors='coerce')
    df["month"]=list(map(lambda date:date.strftime(format="%B"),df["effective_to_date"]))
    return df

In [ ]:
def transform(df):
    df=df[df['income']>0]
    df["total_claim_amount"]=round(df["total_claim_amount"],0)
    df["customer_lifetime_value"]=round(df["customer_lifetime_value"],0)
    df.vehicle_class[customers_df.vehicle_class.isin(["Luxury Car","Luxury SUV"])] = "Luxury Car"
    df.vehicle_class[customers_df.vehicle_class.isin(["Four-Door Car","SUV"])] = "Four-Door Car"
    df.policy[df.policy.isin(["Personal L3","Personal L2","Personal L1"]) ] = "Personal"
    df.policy[df.policy.isin(["Corporate L3","Corporate L2","Corporate L1"]) ] = "Corporate"
    df.policy[df.policy.isin(["Special L3","Special L2","Special L1"]) ] = "Special"
    df=df.drop(columns=["month","education"])
    df.number_of_policies[df.number_of_policies>2]=3
    return df

In [ ]:
customers_df=load_original_data()
#customers_df=clean_data(customers_df)
#customers_df=transform(customers_df)

customers_df = (                  
clean_data(customers_df)                 
.pipe(transform) 
)

In [ ]:
customers_df.shape

In [ ]:
customers_df.columns

X-Y split  
splitting the target variable from the dependent variables

In [ ]:
X=customers_df.drop('total_claim_amount', axis=1)
y=customers_df.total_claim_amount

# we drop some useless columns
X=X.drop(columns=['effective_to_date'])

Get the numeric data.

In [ ]:
numericals =X._get_numeric_data()
# numericals =data.select_dtypes(np.number)
numericals.head()

In [ ]:
numericals.describe()

#### Checking Multicorrelinilaty

In [ ]:
correlations = numericals.corr()
print(correlations)
cmap = sns.cm.rocket_r  ## for reversing the colors
mask=np.zeros_like(numericals.corr())
mask[np.triu_indices_from(mask)]=True
fig, ax=plt.subplots(figsize=(10, 8))
ax=sns.heatmap(numericals.corr(), mask=mask, annot=True,cmap=cmap)
plt.show()

There is no multicollinearity between the numerical features, highest covarriance is 0.4.

#### Standardize

In [ ]:
# Before starting with the linear regression model we standardize the data for better model fit.
X_scaled=StandardScaler().fit_transform(numericals)
X_scaled.shape

### Get Categorical data

In [ ]:
# Getting categorical data

categoricals_features = X.select_dtypes(object)

### Categorical Variables Transformation

One Hot Encoding

In [ ]:
# get dummies pandas
categoricals_features=pd.get_dummies(categoricals_features, drop_first=True)
##pd.DataFrame(OneHotEncoder(drop='first').fit_transform(categoricals_features).toarray(),
## columns=OneHotEncoder(drop='first').fit(categoricals_features).get_feature_names(input_features=categoricals_features.columns)).head()

In [ ]:
categoricals_features

In [ ]:
all_features=pd.concat([numericals, categoricals_features], axis=1) # concat numerical and categorical transformations
all_features.head()

### Train Test Split

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(all_features, y, test_size=.20)
X_train

### Apply Linear Regression

In [ ]:
# Linear regression / OLS regression

model=LinearRegression()    # model
model.fit(X_train.to_numpy(), y_train)   # model train

y
y_pred=model.predict(X_test.to_numpy())   # model prediction

y_pred_train=model.predict(X_train.to_numpy())

In [ ]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression, KNeihgbors

dict_classifiers = {
    "Lasso": Lasso(),
    "clf": Ridge(alpha=1.0),
    "elastic_net" : ElasticNet(random_state=0)
} 

def model_check(dict_classifiers):
    for modeltype, model_instantiation in dict_classifiers():
        model=model_instantiation
        model.fit(X_train, y_train)
        print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")
      
#dict_classifiers = {
#    "Logreg": LogisticRegression(solver='lbfgs'),
#    "NN": KNeighborsClassifier(),
#    "LinearSVM": SVC(probability=True, kernel='linear'), #class_weight='balanced'
#    "GBC": GradientBoostingClassifier(),
#    "DT": tree.DecisionTreeClassifier(),
#    "RF": RandomForestClassifier(),
#    "NB": GaussianNB(),
#}

#for model, model_instantiation in dict_classifiers.iteritems():
#     y_score = model_instantiation.fit(X_train, y_train).decision_function(X_test)

In [ ]:
import this

In [ ]:
for i, in element in enumerate ([2,3,5,7,9])

In [ ]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression

In [ ]:
model_dict = {
    "Lasso": Lasso(),
    "clf": Ridge(alpha=1.0),
    "elastic_net" : ElasticNet(random_state=0)
    }

In [ ]:
def eval_model(X, y, model_dct):
    
    for name, model in model_dict.items():

In [ ]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

In [ ]:
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model=Lasso()
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")
model=Ridge(alpha=1.0)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")
model=ElasticNet(random_state=0)
model.fit(X_train, y_train)
print(f"{model.__class__.__name__}: Train -> {model.score(X_train, y_train)}, Test -> {model.score(X_test, y_test)}")

#### Model Validation

MSE.

In [ ]:
train_mse=mse(y_train,model.predict(X_train.to_numpy()))
test_mse=mse(y_test,model.predict(X_test.to_numpy()))

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

RMSE.

In [ ]:
print ('train RMSE: {} -- test RMSE: {}'.format(train_mse**.5, test_mse**.5))

MAE.

In [ ]:
train_mae=mae(y_train,model.predict(X_train.to_numpy()))
test_mae=mae(y_test,model.predict(X_test.to_numpy()))

print ('train MAE: {} -- test MAE: {}'.format(train_mse, test_mse))

R2.

In [ ]:
r2= model.score(X_test.to_numpy(), y_test)
print (r2)

print(r2_score(y_test,model.predict(X_test.to_numpy())))

In [ ]:

print ('train R2: {} -- test R2: {}'.format(model.score(X_train.to_numpy(), y_train),
                                            model.score(X_test.to_numpy(), y_test)))

adjusted rsquare

In [ ]:
N=X_train.shape[0]
p=36
x = (1-r2)
y = (N-1) / (N-p-1)
adj_rsquared = (1 - (x * y))
print("Adjusted-R2 : " , adj_rsquared)

#### Feature Importance

In [ ]:
features_importances = pd.DataFrame(data={
    'Attribute': X_train.columns,
    'Importance': model.coef_
})
features_importances = features_importances.sort_values(by='Importance', ascending=False)

In [ ]:
## decide on each variable importance on your model using adjusted R_squared 

In [ ]:
features_importances

In [ ]:
plt.bar(x=features_importances['Attribute'], height=features_importances['Importance'], color='#087E8B')
plt.title('Feature importances obtained from coefficients', size=20)
plt.xticks(rotation='vertical')
plt.show()

#### evaluating the importance of new variable to my study  
#### pseduo code
for each iteration
    * X_train(including new variable) = 
    * P= number of paramter 
    * new adjusted_R2= compute adjusted_r2 (p)
    * if adjusted_r_squared_with_new_feature <=  adjusted_r_squared_without_new_feature 
        drop the new variable because it is not significant to my study 

In [ ]:
# Make an scatter plot y_pred vs y
# What kind of plot you will get if all the all the predictions are ok?
# A stright line

fig, ax = plt.subplots(2,2)
ax[0,0].plot(y_pred, y_test, 'o')
ax[0,0].set_xlabel("y_test")
ax[0,0].set_ylabel("y_pred")
ax[0,0].set_title("Test Set -Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[0,1].hist(y_test - y_pred)
ax[0,1].set_xlabel("Test y-y_pred")
ax[0,1].set_title("Test Set Residual histogram")

ax[1,0].plot(y_pred_train, y_train, 'o')
ax[1,0].set_xlabel("y_train")
ax[1,0].set_ylabel("y_pred_train")
ax[1,0].set_title("Train set Predicted vs real")

# Get a histogram of the residuals ie: y - y_pred.  Homoscdasticity
# It resembles a normal distribution?
ax[1,1].hist(y_train - y_pred_train)
ax[1,1].set_xlabel("Train y-y_pred")
ax[1,1].set_title("Train Residual histogram")

# Plotting

EDA Catgorical Columns 

In [ ]:
for c in customers_df.select_dtypes(object):
    plt.figure(figsize=(10,5))
    plt.bar(customers_df[c].unique(), customers_df[c].value_counts())
    plt.title(c)
    plt.show();

### Show a plot of the total number of responses.

In [ ]:
customers_df.customer_lifetime_value.hist()

In [ ]:
sns.countplot('response', data=customers_df)
plt.ylabel('Total number of Response')
plt.show()

### Show a plot of the response rate by the sales channel.

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot('response', hue='sales_channel', data=customers_df)
plt.ylabel('Response by Sales Channel')
plt.show()


### Show a plot of the response rate by the total claim amount.


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y='total_claim_amount' , x='response', data=customers_df)
plt.ylabel('Response by Total Claim Amount')
plt.show()


### Show a plot of the response rate by income.


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(y='income' , x='response', data=customers_df)
plt.ylabel('Response by Income')
plt.show()

# Data Transformation Lab

#### Check normality visually.

In [ ]:
sns.pairplot(customers_df)
plt.show()

In [ ]:
sns.distplot((customers_df['customer_lifetime_value']), bins=20)

In [ ]:
monthly=boxcox(customers_df['customer_lifetime_value'],lmbda=-0.04)

In [ ]:
sns.histplot(monthly)

In [ ]:
sns.distplot((customers_df['monthly_premium_auto']), bins=20)

In [ ]:
pt = PowerTransformer()
monthly=pt.fit_transform(customers_df['monthly_premium_auto'].to_numpy().reshape(-1,1))
sns.displot(monthly)

In [ ]:
monthly,lam=boxcox(customers_df['monthly_premium_auto'])
sns.histplot(monthly)

In [ ]:
qt = QuantileTransformer()
monthly=qt.fit_transform(customers_df['monthly_premium_auto'].to_numpy().reshape(-1,1))
sns.histplot(monthly)

In [ ]:
sns.regplot(x='monthly_premium_auto',y='total_claim_amount', data=customers_df, scatter_kws={"color": "olive"}, line_kws={"color": "black"})